In [1]:
import numpy as np
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings('ignore')

In [2]:
# Step 1: Create an imbalanced binary classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=2, n_redundant=8, 
                           weights=[0.9, 0.1], flip_y=0, random_state=42)

np.unique(y, return_counts=True)

(array([0, 1]), array([900, 100], dtype=int64))

In [3]:
# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)

In [4]:
from imblearn.combine import SMOTETomek

smt = SMOTETomek(random_state=42)
X_train_res, y_train_res = smt.fit_resample(X_train, y_train)


In [5]:
models = [
    (
        "Logistic Regression", 
        {"C": 1, "solver": 'liblinear'},
        LogisticRegression(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "Random Forest", 
        {"n_estimators": 30, "max_depth": 3},
        RandomForestClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train, y_train),
        (X_test, y_test)
    ),
    (
        "XGBClassifier With SMOTE",
        {"use_label_encoder": False, "eval_metric": 'logloss'},
        XGBClassifier(), 
        (X_train_res, y_train_res),
        (X_test, y_test)
    )
]

In [6]:
reports = []

for model_name, params, model, train_set, test_set in models:
    X_train = train_set[0]
    y_train = train_set[1]
    X_test = test_set[0]
    y_test = test_set[1]
    
    model.set_params(**params)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    report = classification_report(y_test, y_pred, output_dict=True)
    reports.append(report)

In [7]:
import mlflow
import mlflow.sklearn
import mlflow.xgboost

In [11]:
# Initialize MLflow
mlflow.set_experiment("Anomaly Detectionn")
mlflow.set_tracking_uri("http://localhost:5000")

for i, element in enumerate(models):
    model_name = element[0]
    params = element[1]
    model = element[2]
    report = reports[i]
    
    with mlflow.start_run(run_name=model_name):        
        mlflow.log_params(params)
        mlflow.log_metrics({
            'accuracy': report['accuracy'],
            'recall_class_1': report['1']['recall'],
            'recall_class_0': report['0']['recall'],
            'f1_score_macro': report['macro avg']['f1-score']
        })  
        
        if "XGB" in model_name:
            mlflow.xgboost.log_model(model, "model")
        else:
            mlflow.sklearn.log_model(model, "model")  

2025/04/04 19:41:12 INFO mlflow.tracking.fluent: Experiment with name 'Anomaly Detectionn' does not exist. Creating a new experiment.
2025/04/04 19:41:24 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Logistic Regression at: http://localhost:5000/#/experiments/576487519231268589/runs/7bee3016ef044c80b72f2d0cf7daa952
🧪 View experiment at: http://localhost:5000/#/experiments/576487519231268589


2025/04/04 19:41:32 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run Random Forest at: http://localhost:5000/#/experiments/576487519231268589/runs/246d5386e07a4fc59397239e49a00ffd
🧪 View experiment at: http://localhost:5000/#/experiments/576487519231268589


2025/04/04 19:41:41 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier at: http://localhost:5000/#/experiments/576487519231268589/runs/5d1dea8d3d444dafbf51a7f73d224ad4
🧪 View experiment at: http://localhost:5000/#/experiments/576487519231268589


2025/04/04 19:41:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/576487519231268589/runs/01dc1a5d4cc842c9bfd91222a1369210
🧪 View experiment at: http://localhost:5000/#/experiments/576487519231268589


Register the Model

In [12]:
model_name = 'XGB-Smote'
run_id=input('Please type RunID')
model_uri = f'runs:/{run_id}/model'

with mlflow.start_run(run_id=run_id):
    mlflow.register_model(model_uri=model_uri, name=model_name)

Registered model 'XGB-Smote' already exists. Creating a new version of this model...
2025/04/04 19:43:42 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: XGB-Smote, version 4


🏃 View run XGBClassifier With SMOTE at: http://localhost:5000/#/experiments/576487519231268589/runs/01dc1a5d4cc842c9bfd91222a1369210
🧪 View experiment at: http://localhost:5000/#/experiments/576487519231268589


Created version '4' of model 'XGB-Smote'.


Load the Model

In [13]:
import mlflow.xgboost
import pandas as pd

model_name = "XGB-Smote"  # replace with your registered model name
model_uri = f"models:/{model_name}@challenger"  # or use @Production, @Staging, etc.

# Load the model
loaded_model = mlflow.xgboost.load_model(model_uri)

# Make predictions (assuming X_test is your feature set)
y_pred = loaded_model.predict(X_test)

print(y_pred[:5])


MlflowException: The following failures occurred while downloading one or more artifacts from http://localhost:5000/api/2.0/mlflow-artifacts/artifacts/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name:
##### File  #####
API request to http://localhost:5000/api/2.0/mlflow-artifacts/artifacts/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name/ failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name/ (Caused by ResponseError('too many 500 error responses'))

In [15]:
# model_version = 2
model_uri = "runs:/01dc1a5d4cc842c9bfd91222a1369210/model"  # replace "model" with actual artifact folder name if needed

loaded_model = mlflow.xgboost.load_model(model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

array([0, 0, 0, 0])

In [14]:
model_uri = "runs:/01dc1a5d4cc842c9bfd91222a1369210/model"  # replace "model" with actual artifact folder name if needed
loaded_model = mlflow.xgboost.load_model(model_uri)

Transition the Model to Production

In [21]:
from mlflow.tracking import MlflowClient

client = MlflowClient()

for mv in client.search_model_versions(f"name='{model_name}'"):
    print(f"Version: {mv.version}, Stage: {mv.current_stage}, Source: {mv.source}")


Version: 4, Stage: None, Source: mlflow-artifacts:/576487519231268589/01dc1a5d4cc842c9bfd91222a1369210/artifacts/model
Version: 3, Stage: None, Source: mlflow-artifacts:/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model
Version: 2, Stage: None, Source: mlflow-artifacts:/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name
Version: 1, Stage: None, Source: mlflow-artifacts:/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name


In [22]:
production_model_name = "anomaly-detection-prod"
current_model_uri = f"models:/{production_model_name}@champion"


client = mlflow.MlflowClient()
client.copy_model_version(src_model_uri=current_model_uri, dst_name=production_model_name)

Registered model 'anomaly-detection-prod' already exists. Creating a new version of this model...
Copied version '1' of model 'anomaly-detection-prod' to version '2' of model 'anomaly-detection-prod'.


<ModelVersion: aliases=[], creation_timestamp=1743779872076, current_stage='None', description='', last_updated_timestamp=1743779872076, name='anomaly-detection-prod', run_id='6c0c3b52c70642359b52d49f9afe4578', run_link='', source='models:/anomaly-detection-prod/1', status='READY', status_message=None, tags={}, user_id='', version='2'>

In [23]:
model_version = 1
prod_model_uri = f"models:/{production_model_name}@champion"

loaded_model = mlflow.xgboost.load_model(prod_model_uri)
y_pred = loaded_model.predict(X_test)
y_pred[:4]

MlflowException: The following failures occurred while downloading one or more artifacts from http://localhost:5000/api/2.0/mlflow-artifacts/artifacts/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name:
##### File  #####
API request to http://localhost:5000/api/2.0/mlflow-artifacts/artifacts/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name/ failed with exception HTTPConnectionPool(host='localhost', port=5000): Max retries exceeded with url: /api/2.0/mlflow-artifacts/artifacts/487562323121499508/6c0c3b52c70642359b52d49f9afe4578/artifacts/model_name/ (Caused by ResponseError('too many 500 error responses'))